**Download and unpack data from Kaggle**

In [1]:
!pip install kaggle

In [0]:
import os
user = ''
key = ''
 
if '.kaggle' not in os.listdir('/root'):
    !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c histopathologic-cancer-detection

In [0]:
!unzip train.zip -d train

In [0]:
!unzip test.zip -d test

In [0]:
!unzip train_labels.csv.zip

In [133]:
!unzip sample_submission.csv.zip

Archive:  sample_submission.csv.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: sample_submission.csv   


**Preparing data**

In [0]:
# Libraries
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [0]:
## Parameters for model

# Hyper parameters
num_epochs = 8
num_classes = 2
batch_size = 128
learning_rate = 0.002

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
labels = pd.read_csv('train_labels.csv')
sub = pd.read_csv('sample_submission.csv')
train_path = 'train'
test_path = 'test'

In [92]:
train, val = train_test_split(labels, stratify=labels.label, test_size=0.1)
len(train), len(val)

(198022, 22003)

*Simple custom generator*

In [0]:
class MyDataset(Dataset):
    def __init__(self, df_data, data_dir = './', transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_name,label = self.df[index]
        img_path = os.path.join(self.data_dir, img_name+'.tif')
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [0]:
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

dataset_train = MyDataset(df_data=train, data_dir=train_path, transform=trans_train)
dataset_valid = MyDataset(df_data=val, data_dir=train_path, transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)


**Model**

****Public Score 0.9539****

In [0]:
# NOTE: CLASS IS INHERITED FROM nn.Module
class SimpleConvNet(nn.Module):
    def __init__(self):
        # ancestor constructor call
        super(SimpleConvNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.fc = nn.Linear(64 * 29 * 29, 2)  # !!!

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        # print(x.shape) lifehack to find out the desired dimension for the Liner layer
        x = x.view(-1, 64 * 29 * 29)  # !!!
        x = self.fc(x)
        return x

Important note: You may notice that in lines with # !!! there is not very clear 64 * 29 *29. This is the dimension of the picture before the FC layers (H x W x C), then you have to calculate it manually (in Keras, for example, .Flatten () does everything for you). However, there is one life hack — just make print (x.shape) in forward () (commented out line). You will see the size (batch_size, C, H, W) - you need to multiply everything except the first (batch_size), this will be the first dimension of Linear (), and it is in C H W that you need to "expand" x before feeding to Linear ().

In [0]:
model = SimpleConvNet().to(device)

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)


In [101]:

# Train the model
total_step = len(loader_train)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/8], Step [100/1548], Loss: 0.5498
Epoch [1/8], Step [200/1548], Loss: 0.4306
Epoch [1/8], Step [300/1548], Loss: 0.5542
Epoch [1/8], Step [400/1548], Loss: 0.4787
Epoch [1/8], Step [500/1548], Loss: 0.5290
Epoch [1/8], Step [600/1548], Loss: 0.4210
Epoch [1/8], Step [700/1548], Loss: 0.4518
Epoch [1/8], Step [800/1548], Loss: 0.4656
Epoch [1/8], Step [900/1548], Loss: 0.4175
Epoch [1/8], Step [1000/1548], Loss: 0.4213
Epoch [1/8], Step [1100/1548], Loss: 0.6913
Epoch [1/8], Step [1200/1548], Loss: 0.6548
Epoch [1/8], Step [1300/1548], Loss: 0.4138
Epoch [1/8], Step [1400/1548], Loss: 0.5629
Epoch [1/8], Step [1500/1548], Loss: 0.3609
Epoch [2/8], Step [100/1548], Loss: 0.3705
Epoch [2/8], Step [200/1548], Loss: 0.3680
Epoch [2/8], Step [300/1548], Loss: 0.3773
Epoch [2/8], Step [400/1548], Loss: 0.3150
Epoch [2/8], Step [500/1548], Loss: 0.3285
Epoch [2/8], Step [600/1548], Loss: 0.3779
Epoch [2/8], Step [700/1548], Loss: 0.4119
Epoch [2/8], Step [800/1548], Loss: 0.2917
Epoch

In [102]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model on the 22003 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 22003 test images: 90.18770167704405 %


In [0]:
dataset_valid = MyDataset(df_data=sub, data_dir=test_path, transform=trans_valid)
loader_test = DataLoader(dataset = dataset_valid, batch_size=32, shuffle=False, num_workers=0)

In [0]:

model.eval()

preds = []
for batch_i, (data, target) in enumerate(loader_test):
    data, target = data.cuda(), target.cuda()
    output = model(data)

    pr = output[:,1].detach().cpu().numpy()
    for i in pr:
        preds.append(i)

sub['label'] = preds
sub.to_csv('sub.csv', index=False)

In [138]:
sub.shape, len(preds)

((57458, 2), 57458)

In [0]:
from google.colab import files
files.download('sub.csv')

**Fast Keras**

In [0]:
# Libraries
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adamax
from keras.preprocessing.image import ImageDataGenerator

In [44]:
df = pd.read_csv('train_labels.csv')
df["id"]=df["id"].apply(lambda x : x +".tif")
df['label'] = df['label'].astype('str')
df.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835.tif,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77.tif,1
2,755db6279dae599ebb4d39a9123cce439965282d.tif,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08.tif,0
4,068aba587a4950175d04c680d38943fd488d6a9d.tif,0


In [47]:
# num_val
val = .1

# Generator for data augmentation
datagen= ImageDataGenerator(
    rescale=1./255,
    samplewise_std_normalization= True,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    zoom_range=0.2, 
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    channel_shift_range=0.1,
    validation_split=val)


# Generator for train data
train_generator=datagen.flow_from_dataframe(
    dataframe=df,
    directory='train',
    x_col="id",
    y_col="label",
    subset="training",
    batch_size=128,
    shuffle=True,
    class_mode="binary",
    target_size=(96,96))


# Generator for test data
valid_generator=datagen.flow_from_dataframe(
    dataframe=df,
    directory='train',
    x_col="id",
    y_col="label",
    subset="validation",
    batch_size=128,
    shuffle=True,
    class_mode="binary",
    target_size=(96,96))


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:353: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 198023 images belonging to 2 classes.
Found 22002 images belonging to 2 classes.


**Model**

In [0]:
model = Sequential()
model.add(Conv2D(16, kernel_size=3, input_shape=(96,96,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))


model.add(Conv2D(32, kernel_size=3, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))


model.add(Conv2D(64, kernel_size=3, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))


model.add(Flatten())
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [0]:
adamax = Adamax(lr=0.02,
                beta_1=0.9,
                beta_2=0.999,
                epsilon=None,
                decay=0.0)

In [0]:
model.compile(optimizer=adamax,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [80]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 96, 96, 16)        448       
_________________________________________________________________
batch_normalization_20 (Batc (None, 96, 96, 16)        64        
_________________________________________________________________
activation_26 (Activation)   (None, 96, 96, 16)        0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 48, 48, 16)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 48, 48, 32)        4640      
_________________________________________________________________
batch_normalization_21 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
activation_27 (Activation)   (None, 48, 48, 32)        0         
__________

In [81]:
# Начало обучения
history = model.fit_generator(train_generator,
                              steps_per_epoch=train_generator.n//128, 
                              validation_data=valid_generator,
                              validation_steps=valid_generator.n//128,
                              epochs=5,
                              verbose=1)

Epoch 1/5
1547/1547 [==============================] - 1204s 779ms/step - loss: 6.5364 - acc: 0.5942 - val_loss: 6.4858 - val_acc: 0.5976
Epoch 2/5
1547/1547 [==============================] - 1203s 778ms/step - loss: 6.5303 - acc: 0.5948 - val_loss: 6.5131 - val_acc: 0.5959
Epoch 3/5
1547/1547 [==============================] - 1207s 780ms/step - loss: 6.5287 - acc: 0.5949 - val_loss: 6.5419 - val_acc: 0.5941
Epoch 4/5
1547/1547 [==============================] - 1209s 782ms/step - loss: 6.5190 - acc: 0.5955 - val_loss: 6.5433 - val_acc: 0.5940
Epoch 5/5
1547/1547 [==============================] - 1167s 754ms/step - loss: 6.5260 - acc: 0.5951 - val_loss: 6.4490 - val_acc: 0.5999
